### Layering

<img src="../images/eli.jpg" alt="eli" width="400">

In [1]:
import altair as alt
import pandas as pd

In [2]:
eli = pd.read_csv('../data/skydiving.csv')
eli['ground'] = 0

In [3]:
eli.head()

,jump,exit_alt,deploy_alt,freefall,ground
0,1,13500,6000,7500,0
1,2,13500,6000,7500,0
2,3,14000,6000,8000,0
3,4,13500,6000,7500,0
4,5,11000,6000,5000,0


Can we create and repurpose a candle stick chart?

<img src="../images/candlestick.png" alt="candlestick" width="400">

[Source](https://altair-viz.github.io/gallery/candlestick_chart.html)

In [4]:
alt.Chart(eli).mark_rule().encode(
    x='jump',
    y='deploy_alt',
    y2='exit_alt'
)

alt.Chart(...)

Start small...

In [5]:
eli50 = eli.head(50)

In [6]:
top = alt.Chart(eli50).mark_rule(size=3).encode(
    x='jump',
    y='deploy_alt',
    y2='exit_alt'
)

bottom = alt.Chart(eli50).mark_rule(size=1).encode(
    x='jump',
    y='deploy_alt',
    y2='ground'
)

# abc ... 123
top + bottom

alt.LayerChart(...)

<img src="../images/less.gif" alt="choose" width="400">

In [7]:
top = alt.Chart(eli50).mark_rule(size=3).encode(
    x=alt.X('jump', axis=None),
    y=alt.Y('deploy_alt', axis=None),
    y2=alt.Y2('exit_alt'),
)

bottom = alt.Chart(eli50).mark_rule(size=1).encode(
    x=alt.X('jump', axis=None),
    y=alt.Y('deploy_alt', axis=None),
    y2=alt.Y2('ground')
)

chart = (
    # layer here
    (top + bottom)
    .properties(height=80)
    .configure_axis(grid=False)
    .configure_view(strokeWidth=0)
)

chart

alt.LayerChart(...)

#### Plot Everything

In [8]:
eli['cut'] = eli['jump'] // 50
eli['jump_in_cut'] = eli['jump'] % 50

In [9]:
eli

,jump,exit_alt,deploy_alt,freefall,ground,cut,jump_in_cut
0,1,13500,6000,7500,0,0,1
1,2,13500,6000,7500,0,0,2
2,3,14000,6000,8000,0,0,3
3,4,13500,6000,7500,0,0,4
4,5,11000,6000,5000,0,0,5
...,...,...,...,...,...,...,...
216,217,10500,2330,8170,0,4,17
217,218,10600,2540,8060,0,4,18
218,219,10500,2560,7940,0,4,19
219,220,11000,2350,8650,0,4,20


In [10]:
top = alt.Chart(eli).mark_rule(size=3).encode(
    x=alt.X('jump_in_cut', axis=None),
    y=alt.Y('deploy_alt', axis=None),
    y2=alt.Y2('exit_alt'),
).properties(height=80)

bottom = alt.Chart(eli).mark_rule(size=1).encode(
    x=alt.X('jump_in_cut', axis=None),
    y=alt.Y('deploy_alt', axis=None),
    y2=alt.Y2('ground'),
).properties(height=80)

chart = (
    alt.layer(top, bottom)
    # biggest thing that changed...
    .facet(row = alt.Row("cut", title=None))
    .configure_axis(grid=False)
    .configure_view(strokeWidth=0)
)

chart

alt.FacetChart(...)

### Faceting

ZA5950: International Social Survey Programme: National Identity III - ISSP 2013

Question: "Do Immigrants Increase Crime"?
        
[Source](https://zacat.gesis.org/webview/index.jsp?object=http://zacat.gesis.org/obj/fStudy/ZA5950)

In [11]:
im = pd.read_csv('../data/immigration.csv')

In [12]:
im.sample(5)

,country,response,n,pct
56,Lithuania,Undecided or Neutral,536,44.9
72,Slovak Republic,Agree or Strongly Agree,605,52.3
76,Slovenia,Disagree or Strongly Disagree,283,28.0
70,Russia,Disagree or Strongly Disagree,139,9.2
15,Finland,Agree or Strongly Agree,610,49.1


In [13]:
im['pct'] /= 100

In [14]:
yes = im[im['response'] == 'Agree or Strongly Agree']

In [15]:
alt.Chart(yes).mark_bar().encode(y='country', x='pct')

alt.Chart(...)

In [16]:
chart = (
    alt.Chart(im)
    .mark_bar()
    .encode(
        y='country', 
        x='pct'
    )
    .properties(width=100)
    .facet(column='response')
)

chart

alt.FacetChart(...)

Clean up...

In [17]:
sort = yes.sort_values('pct', ascending=False)['country'].values.tolist()

In [18]:
chart = (
    alt.Chart(im)
    .mark_bar()
    .encode(
        y=alt.Y('country', sort=sort, title=None),
        x=alt.X('pct', axis=alt.Axis(format='%'), title=None),
        color=alt.Color(
            'response', 
            legend=None, 
            scale=alt.Scale(range=["#1c77c3", "#39a9db", "#8C8C8C"]))
    )
    .properties(width=150, height=350)
    .facet(column=alt.Column('response', title=''))
)

chart.properties(title="Do Immigrants Increase Crime?")

alt.FacetChart(...)

### Back to `sex.csv`

ZA4950: International Social Survey Programme: Religion III - ISSP 2008
        
[Source](https://zacat.gesis.org/webview/index.jsp?object=http://zacat.gesis.org/obj/fStudy/ZA4950)

In [19]:
sex = pd.read_csv('../data/sex.csv')
sex = sex.rename(columns={'age2': 'age'})
sex['pct'] /= 100

In [20]:
sex.sample(5)

,age,religion,response,n,pct
173,60-69,Hinduism,Wrong-ish,1,0.037
156,50-59,Other Religions,Not wrong,22,0.265
213,70+,No religion,Wrong,236,0.262
169,60-69,Christian Orthodox,Wrong,272,0.637
55,25-29,Other Eastern Religions,Wrong,21,0.208


Where we left off...

In [21]:
no = sex[sex['religion'] == 'No religion']

chart = (
    alt.Chart(no)
    .mark_circle(opacity=3/4)
    .encode(
        x=alt.X('pct:Q', axis=alt.Axis(title='', format='%')),
        y=alt.Y('age', 
            axis=alt.Axis(title='', grid=True),
            scale=alt.Scale(domain=['70+','60-69', '50-59', '40-49', '30-39', '25-29', '18-24']),
        ), 
        color=alt.Color('response', 
            scale=alt.Scale(
                domain=['Not wrong', 'Wrong-ish', 'Wrong'], 
                range=["#39a9db", "#f39237", "#d63230"]),
            legend=alt.Legend(title='', orient='top')
        )
    )
)

chart

alt.Chart(...)

In [22]:
chart = (
    alt.Chart(sex)
    .mark_circle(opacity=3/4)
    .encode(
        x=alt.X('pct:Q', axis=alt.Axis(title='', format='%')),
        y=alt.Y('age', 
            axis=alt.Axis(title='', grid=True),
            scale=alt.Scale(domain=['70+','60-69', '50-59', '40-49', '30-39', '25-29', '18-24']),
        ), 
        color=alt.Color('response', 
            scale=alt.Scale(
                domain=['Not wrong', 'Wrong-ish', 'Wrong'], 
                range=["#39a9db", "#f39237", "#d63230"]),
            legend=alt.Legend(title='', orient='top')
        )
    )
    .properties(height=100, width=100)
    .facet('religion', columns=3)
    .configure_view(strokeWidth=0)
    .properties(
        background='#F0F0F0',
        title='Sex Before Marriage'
    )
)

chart

alt.FacetChart(...)

#### Another Layer & Facet Example

In [23]:
cocktails = pd.read_csv('../data/cocktails.tsv', delimiter='\t')

In [24]:
cocktails.sample(3)

,name,abv,acid,sugar,type,index,instructions,ingredients,ncotw
24,Between the Sheets,20.1,0.58,4.5,shaken,29,"Shake, coupe. Express and discard lemon twist.",1 1/2 oz Cognac (41% abv)<br/>3/4 oz Curacao<b...,"<a href=""https://www.reddit.com/r/cocktails/co..."
41,Improved Whiskey Cocktail,29.3,0.00,6.5,stirred,46,"Stir, sever over large rock in absinthe-rinsed...",2 oz rye (50% abv)<br/>1/4 oz Luxardo Maraschi...,"<a href=""https://www.reddit.com/r/cocktails/co..."
7,Blood and Sand,15.6,0.19,8.0,shaken,12,"Shake, coupe. (Optionally flamed) orange twist.",1 oz Scotch (43% abv)<br/>3/4 oz Cherry Herrin...,"<a href=""https://www.reddit.com/r/cocktails/co..."


In [25]:
alt.Chart(cocktails).mark_circle().encode(y='name', x='abv')

alt.Chart(...)

In [26]:
popular = [
    'Whiskey Sour',
    'Margarita',
    'Negroni', 
    'Blender Daiquiri', 
    'Gin and Tonic (Dry)'
]

pop = cocktails[cocktails['name'].isin(popular)]

In [27]:
acid = alt.Chart(pop).mark_circle(color='yellow').encode(
    y='name', x='abv', size='acid'
)

sugar = alt.Chart(pop).mark_circle(color='pink').encode(
    y='name', x='abv', size='sugar'
)

sugar + acid

alt.LayerChart(...)

In [28]:
from sklearn.preprocessing import StandardScaler

cocktails['acid'] = StandardScaler().fit_transform(cocktails[['acid']])
cocktails['sugar'] = StandardScaler().fit_transform(cocktails[['sugar']])

In [29]:
pop = cocktails[cocktails['name'].isin(popular)]

acid = alt.Chart(pop).mark_circle(opacity=1/8, color='blue').encode(
    y='name', x='abv', size='acid'
)

sugar = alt.Chart(pop).mark_point(opacity=1, color='pink').encode(
    y='name', x='abv', size='sugar'
)

(sugar + acid).properties(width=250, height=250, title='Sugar (Pink) & Acid (Blue)')

alt.LayerChart(...)

### Concatenating

In [30]:
marathon = pd.read_csv('../data/marathon.csv')
marathon['time'] = marathon['time'].apply(pd.to_datetime)

In [31]:
two = marathon[marathon['runner'].isin(['Emma S.', 'Rich H.'])]

In [32]:
two.sample(5)

,time,distance,runner,dsplit,mph,hour,gender
393,2019-03-26 04:00:00,93.873060,Rich H.,5.215170,2.607585,34,male
114,2019-03-25 22:00:00,65.189625,Emma S.,2.607585,1.303792,28,female
396,2019-03-26 10:00:00,101.695815,Rich H.,5.215170,2.607585,40,male
386,2019-03-25 14:00:00,67.797210,Rich H.,5.215170,2.607585,20,male
389,2019-03-25 20:00:00,80.835135,Rich H.,2.607585,1.303792,26,male


In [33]:
distance = alt.Chart(two).mark_line().encode(
    x='hour', y='distance', color='runner'
).properties(height=100)

distance

alt.Chart(...)

In [34]:
mph = alt.Chart(two).mark_line().encode(
    x='hour', y='mph', color='runner'
).properties(height=100)

mph

alt.Chart(...)

In [35]:
rolling_mph = alt.Chart(two).mark_line().transform_window(
    rolling_mean='mean(mph)',
    frame=[-2, 2]
).encode(
    x='hour',
    y='rolling_mean:Q',
    color='runner'
).properties(height=100)

rolling_mph

alt.Chart(...)

In [36]:
distance | mph | rolling_mph

alt.HConcatChart(...)

In [37]:
distance & mph & rolling_mph

alt.VConcatChart(...)

### Cheat Sheet

Altair provides a number of compound plot types that can be used to create stacked, layered, faceted, and repeated charts:

| class                                                        | functional form               | operator form     | reference                                                    |
| ------------------------------------------------------------ | ----------------------------- | ----------------- | ------------------------------------------------------------ |
| [`LayerChart`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.LayerChart.html#altair.LayerChart) | `alt.layer(chart1, chart2)`   | `chart1 + chart2` | [Layered Charts](https://altair-viz.github.io/user_guide/compound_charts.html#layer-chart) |
| [`HConcatChart`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.HConcatChart.html#altair.HConcatChart) | `alt.hconcat(chart1, chart2)` | `chart1 \| chart2` | [Horizontal Concatenation](https://altair-viz.github.io/user_guide/compound_charts.html#hconcat-chart) |
| [`VConcatChart`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.VConcatChart.html#altair.VConcatChart) | `alt.vconcat(chart1, chart2)` | `chart1 & chart2` | [Vertical Concatenation](https://altair-viz.github.io/user_guide/compound_charts.html#vconcat-chart) |

| class                                                        | method form                       | reference                                                    |
| ------------------------------------------------------------ | --------------------------------- | ------------------------------------------------------------ |
| [`RepeatChart`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.RepeatChart.html#altair.RepeatChart) | `chart.repeat(row, column)`       | [Repeated Charts](https://altair-viz.github.io/user_guide/compound_charts.html#repeat-chart) |
| [`FacetChart`](https://altair-viz.github.io/user_guide/generated/toplevel/altair.FacetChart.html#altair.FacetChart) | `chart.facet(facet, row, column)` | [Faceted Charts](https://altair-viz.github.io/user_guide/compound_charts.html#facet-chart) |


#### Exercise

Build a layered Altair chart on top of the `cocktails` data set